In [6]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [7]:
import dill
with open('../objects/preprocessor.pkl','rb') as f:
    ctf=dill.load(f)

In [8]:
ctf

,transformers,"[('ohe_geo', ...), ('le_gender', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,drop,'first'
,sparse_output,True


In [9]:
df=pd.read_csv('../Churn_Modelling.csv')

In [13]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [14]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'],inplace=True)

In [15]:
X,y=df.drop(columns=['Exited']),df['Exited']

In [16]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25)

In [17]:
X_train_transformed=ctf.transform(X_train)

In [19]:
# X_train

#### ANN

In [27]:
import tensorflow as tf #For different attributes like losses, optimizers
from tensorflow.keras.models import Sequential #ann
from tensorflow.keras.layers import Dense  #hidden layers
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard   #logginf

In [39]:
# 1. Create the model and pass the layers
# Dense(no of nuerons,activation function,input shape in first hl)
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_transformed.shape[1],)),  # hl1
    Dense(32, activation='relu'),  # hl2
    Dense(1, activation='sigmoid')  # output layer
])

In [40]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,881 (11.25 KB)

 Trainable params: 2,881 (11.25 KB)

 Non-trainable params: 0 (0.00 B)

In [41]:
# 2. There are two ways of providing loss and optmizers to the model --
# >>keyword
# >>tensorflow object

In [42]:
opt=tf.optimizers.Adam(learning_rate=0.01)
loss=tf.losses.BinaryCrossentropy()

In [43]:
# 3.compile model- we can use different metrics parameter also
model.compile(loss=loss,optimizer=opt,metrics=['accuracy'])

In [44]:
# 4. Add callbacks(here 2 : tensorboard,earlystopping)
import datetime
log_dir = 'logs/dir'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tf_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
# histogram_freq=1 means on every epoch we save logs

In [45]:
early_stopping = EarlyStopping(
    patience=6, monitor='val_loss', restore_best_weights=True)

In [46]:
X_test_transformed=ctf.transform(X_test)

In [47]:
X_train.shape,X_train_transformed.shape

((7500, 10), (7500, 11))

In [49]:
history = model.fit(X_train_transformed, y_train, validation_data=(
    X_test_transformed, y_test), callbacks=[early_stopping, tf_callback],epochs=100)

Epoch 1/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8572 - loss: 0.3526 - val_accuracy: 0.8576 - val_loss: 0.3644
Epoch 2/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8529 - loss: 0.3471 - val_accuracy: 0.8576 - val_loss: 0.3632
Epoch 3/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8637 - loss: 0.3343 - val_accuracy: 0.8564 - val_loss: 0.3620
Epoch 4/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8615 - loss: 0.3351 - val_accuracy: 0.8568 - val_loss: 0.3667
Epoch 5/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8565 - loss: 0.3366 - val_accuracy: 0.8580 - val_loss: 0.3562
Epoch 6/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8612 - loss: 0.3356 - val_accuracy: 0.8552 - val_loss: 0.3694
Epoch 7/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8694 - loss: 0.3173 - val_accuracy: 0.8548 - val_loss: 0.3591
Epoch 8/100
235/235 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8643 - loss: 0.3270 - val_accu

In [50]:
model.save('model.h5')

In [51]:
#Load tensorboard extension for visualization
%load_ext tensorboard

In [57]:
%tensorboard --logdir=logs

In [ ]:
# check http://localhost:6006/

In [ ]:
# prediction-streamlit app